In [1]:
def extract_data_from_table1(table):
    td = table.select_one("tr > td")
    img_src = td.select_one("img")
    result = {
        "ci_market_separation":img_src.get("src", ""),
        "ci_name":td.select_one(".view_tit").text,
        "ci_code":td.select_one(".view_txt01").text
        }


    return result


def extract_data_from_table2(table):
    keys = [
        "ci_ceo",
        "ci_establishment_date",
        "ci_company_separation",
        "ci_brn",
        "ci_tel",
        "ci_homepage",
        "ci_settlement_month",
        "ci_worker_cnt",
        "ci_industries",
        "ci_important_product",
        "ci_stocks_separation",
        "ci_lead_manager",
        "ci_address",
    ]

    result = []
    trs = table.select("tr")
    for idx_tr, tr in enumerate(trs):
        tds = tr.select("td.txt")
        result.extend(td.text if td.text is not None else "" for td in tds)
    result = dict(zip(keys, result))

    return result


def extract_data_from_table3(table):
    keys = [
        "ci_review_c_date",
        "ci_review_a_date",
        "ci_turnover",
        "ci_before_corporate_tax",
        "ci_net_profit",
        "ci_capital",
        "ci_largest_shareholder",
        "ci_largest_shareholder_rate",
        "ci_po_expected_price",
        "ci_po_expected_stocks",
        "ci_po_expected_amount",
        "ci_listing_expected_stocks",
    ]
    tds = table.select('tr > td[width="240"]')
    result = [td.string.strip() if td.string is not None else "" for td in tds]
    return dict(zip(keys, result))



In [ ]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)
import requests
import sys


%autoawait 

In [3]:


from bs4 import BeautifulSoup
from collections import ChainMap



def scrape_ipostock(code):

    url = f"http://www.ipostock.co.kr/view_pg/view_01.asp?code={code}"



    req = requests.get(url)
    soup = BeautifulSoup(req.content, "lxml", from_encoding="utf-8")
    table1 = soup.find("table", width="550", style="margin:0 auto;")
    result = table1.select_one('td > table > tr:nth-child(1) > td > table > tr > td:nth-child(1) > table > tr:nth-child(1) > td > table > tr > td:nth-child(2) > font > b')
    result = soup.select_one('#print > table > tr:nth-child(3) > td > table > tr:nth-child(1) > td > table > tr > td:nth-child(1) > table > tr:nth-child(1) > td > table > tr > td:nth-child(2) > font > b')
    # print(result)
    table2, table3 = soup.select('table[width="780"][class="view_tb"]')





if __name__ == "__main__":
    # 바이오 노트
    code = "B202206162"
    # 래몽래인
    # 밀리의서재
    code = "B202205272"
    code = 'B202106072'
    result = scrape_ipostock(code)

    # from schemas.general import GeneralCreateSchema

    # g = GeneralCreateSchema(**result)
    # from pprint import pprint as pp

    # pp(g.dict())


In [2]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)
import requests
import sys


from bs4 import BeautifulSoup
from collections import ChainMap


code = 'B201803291'
url = f"http://www.ipostock.co.kr/view_pg/view_01.asp?code={code}"



req = requests.get(url)
soup = BeautifulSoup(req.content, "lxml", from_encoding="utf-8")
table1 = soup.find("table", width="550", style="margin:0 auto;")
result = table1.select_one('td > table > tr:nth-child(1) > td > table > tr > td:nth-child(1) > table > tr:nth-child(1) > td > table > tr > td:nth-child(2) > font > b')
result = soup.select_one('#print > table > tr:nth-child(3) > td > table > tr:nth-child(1) > td > table > tr > td:nth-child(1) > table > tr:nth-child(1) > td > table > tr > td:nth-child(2) > font > b')
# print(result)
table2, table3 = soup.select('table[width="780"][class="view_tb"]')
table3



<table class="view_tb" width="780">
<tr>
<td class="tit" width="125">심사청구일</td>
<td class="txt2" width="240">2021.06.24</td>
<td class="tit" width="125">심사승인일</td>
<td class="txt2" width="240">2021.10.01</td>
</tr>
<tr>
<td class="tit" width="125">청구시 매출액</td>
<td class="txt2" width="240">18,260 백만원</td>
<td class="tit" width="125">법인세차감전순이익</td>
<td class="txt2" width="240">3,078 백만원</td>
</tr>
<tr>
<td class="tit" width="125">청구시 순이익</td>
<td class="txt2" width="240">2,916 백만원</td>
<td class="tit" width="125">청구시 자기자본</td>
<td class="txt2" width="240">45,928 백만원	</td>
</tr>
<tr>
<td class="tit" width="125">청구시 최대주주</td>
<td class="txt2" width="240">정한섭</td>
<td class="tit" width="125">최대 주주비율</td>
<td class="txt2" width="240">61 %</td>
</tr>
<tr>
<td class="tit" width="125">공모(예정)발행가</td>
<td class="txt2" width="240"><strong>0 원 ~ 0 원</strong></td>
<td class="tit" width="125">공모(예정)주식수</td>
<td class="txt2" width="240"><strong class="col_red2">1,220,000 주</strong></td>
</tr>
<tr>
<td

In [4]:
keys = [
    "ci_review_c_date",
    "ci_review_a_date",
    "ci_turnover",
    "ci_before_corporate_tax",
    "ci_net_profit",
    "ci_capital",
    "ci_largest_shareholder",
    "ci_largest_shareholder_rate",
    "ci_po_expected_price",
    "ci_po_expected_stocks",
    "ci_po_expected_amount",
    "ci_listing_expected_stocks",
]
tds = table3.select('tr > td[width="240"]')
for td in tds:
    print(td.string.strip())
# result = [td.text if td.text is not None else "" for td in tds]
# return dict(zip(keys, result))

2021.06.24
2021.10.01
18,260 백만원
3,078 백만원
2,916 백만원
45,928 백만원
정한섭
61 %
0 원 ~ 0 원
1,220,000 주
0 백만원 ~ 0 백만원
7,254,894 주


In [10]:
# 바이오 노트
code = "B202206162"
# 래몽래인
code = "B202010131"
url = "http://www.ipostock.co.kr/view_pg/view_01.asp?code={code}&gmenu="
result = scrape_ipostock(url)

# from schemas.general import GeneralCreateSchema

# g = GeneralCreateSchema(**result)
# from pprint import pprint as pp

# pp(g)


None


In [10]:
import sys
from pathlib import Path
project_path = str(Path.cwd().parent)
sys.path.append(project_path)
import requests
import sys


import asyncio
import aiohttp

%autoawait asyncio

In [15]:
from typing import Dict, Union

from bs4 import BeautifulSoup
from async_retrying import retry
from apps.ipo.agents import get_user_agents
from config.config_log import logging

logger = logging.getLogger("info-logger")


async def face_value(soup: BeautifulSoup) -> str:
    """
    Returns the face value of a stock from the given BeautifulSoup object
    """
    ci_face_value = None
    raw_soup = soup.select_one('table[width="554]')
    print(raw_soup)
    tds = soup.select('table[width="390"] td')
    # print(tds)
    if not tds:
        logger.error("액면가를 가져 올수 없습니다. html tag를 확인하세요.")
    for idx, td in enumerate(tds):
        font = td.select_one("font")
        if font and font.get_text() == "액면가":
            ci_face_value = tds[idx + 1].get_text().replace("원", "").replace(",", "").strip()
            break
    return ci_face_value or "0"


async def extract_data_from_table1(table: BeautifulSoup) -> Dict[str, Union[str, int]]:
    """
    Extracts data from the first table of the provided BeautifulSoup object and returns a dictionary
    """
    try:
        data2 = table.select_one(".view_tit").get_text()
        data3 = table.select_one(".view_txt01").get_text()
        data1 = table.select_one("img")["src"]
        return {
            "ci_market_separation": data1,
            "ci_list_type": data1,
            "ci_name": data2,
            "ci_code": data3,
        }
    except KeyError as err:
        logger.error(err)
    except AttributeError as err:
        logger.error(err)


async def extract_data_from_table2(table: BeautifulSoup) -> Dict[str, str]:
    """
    Extracts data from the second table of the provided BeautifulSoup object and returns a dictionary
    """
    keys = [
        "ci_ceo",
        "ci_establishment_date",
        "ci_company_separation",
        "ci_brn",
        "ci_tel",
        "ci_homepage",
        "ci_settlement_month",
        "ci_worker_cnt",
        "ci_industries",
        "ci_important_product",
        "ci_stocks_separation",
        "ci_lead_manager",
        "ci_address",
    ]

    result = []
    trs = table.select("tr")
    try:
        if not trs:
            logger.error("html 태그에 접근 할 수 없습니다.")
        for tr in trs:
            tds = tr.select("td.txt")
            result.extend(td.text if td.text is not None else "" for td in tds)
        result = dict(zip(keys, result))
        return result
    except AttributeError as err:
        logger.error(err)


async def extract_data_from_table3(table: BeautifulSoup) -> Dict[str, str]:
    """
    Extracts data from the third table of the provided BeautifulSoup object and returns a dictionary
    """
    keys = [
        "ci_review_c_date",
        "ci_review_a_date",
        "ci_turnover",
        "ci_before_corporate_tax",
        "ci_net_profit",
        "ci_capital",
        "ci_largest_shareholder",
        "ci_largest_shareholder_rate",
        "ci_po_expected_price",
        "ci_po_expected_stocks",
        "ci_po_expected_amount",
        "ci_listing_expected_stocks",
    ]
    try:
        tds = table.select('tr > td[width="240"]')
        result = [td.string.strip() if td.string is not None else "" for td in tds]
        result = dict(zip(keys, result))
        return result
    except AttributeError as err:
        logger.error(err)


@retry(attempts=100)
async def scrape_ipostock(code: str) -> Dict[str, Union[str, Dict[str, str]]]:
    """
    Scrapes data from ipostock website using the provided code and returns a dictionary of the extracted data
    """
    header = await get_user_agents()
    url = f"http://www.ipostock.co.kr/view_pg/view_01.asp?code={code}"
    # try:
    async with aiohttp.ClientSession() as session:
        async with session.get(url, headers=header) as resp:
            soup = BeautifulSoup(await resp.text(), "lxml")

        td = soup.select_one('table[width="119"] > tr > td')
        img = td.select_one('a > img').get('src')
        if img is None or img == '':
            print('None 혹은 처리할 수 없는 데이터')
            print(img)
        print(img)
    #     table1 = soup.find("table", width="550", style="margin:0 auto;")
    #     table2, table3 = soup.select('table[width="780"][class="view_tb"]')

    #     if not table1 or not table2 or not table3:
    #         logger.error("html 태그 속성을 통해 데이터를 파싱 할 수 없습니다. ")

    #     face_value_result, t1, t2, t3 = await asyncio.gather(
    #         face_value(soup),
    #         extract_data_from_table1(table1),
    #         extract_data_from_table2(table2),
    #         extract_data_from_table3(table3),
    #     )

    #     result = {"ci_face_value": face_value_result, **t1, **t2, **t3}
    #     return result
    # except (aiohttp.ClientError, asyncio.TimeoutError) as e:
    #     logger.error("Request failed, retrying in 5 seconds...")
    #     logger.error(e)
    #     await asyncio.sleep(1)



code = 'B202106072'
await scrape_ipostock(code)



http://www.ipostock.co.kr/upload/img/corp/202167(1).png


In [12]:
async def hello():
    return 'hello'

await hello()

'hello'